# Fill from Outflow Points Example

In [1]:
import heapq
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal

import fill

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10, 10)

### Build a test array

In [2]:
dem_adj = np.array([
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 0.1, 1.0, 1.0, 1.0, 1.0],
    [0.1, 0.6, 0.2, 0.3, 1.0, 1.0],
    [0.2, 0.7, 0.3, 0.4, 0.1, 1.0],
    [1.0, 0.8, 0.4, 1.0, 1.0, 1.0],
    [1.0, 1.0, 0.2, 1.0, 1.0, 1.0],
])
hru_type = np.array([
    [0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 0],
])
outflow_pts = [[3, 4]]
print(dem_adj)
print(hru_type)
print(dem_adj[outflow_pts[0][0], outflow_pts[0][1]])

[[ 1.   1.   1.   1.   1.   1. ]
 [ 1.   0.1  1.   1.   1.   1. ]
 [ 0.1  0.6  0.2  0.3  1.   1. ]
 [ 0.2  0.7  0.3  0.4  0.1  1. ]
 [ 1.   0.8  0.4  1.   1.   1. ]
 [ 1.   1.   0.2  1.   1.   1. ]]
[[0 0 0 0 0 0]
 [0 1 1 1 1 0]
 [0 1 1 1 1 0]
 [0 1 1 1 1 0]
 [0 1 1 1 1 0]
 [0 1 1 1 1 0]]
0.1


In [3]:
def outflow_fill(input_array, outflow_pts, four_way=False):
    """
    Flood fill depressions/sinks in floating point array from outflow points


    Parameters
    ----------
    input_array : ndarray
        Input array to be filled
    outflow_pts : list
        List of outflow array coordinates (row, col)
    four_way : bool, optional
        If True, search 4 immediately adjacent cells
            (cross structuring element)
        If False, search all 8 adjacent cells
            (square structuring element).
        The Default is False.

    Returns
    -------
    out : ndarray
        Filled array

    References
    ----------
    .. [1] Soille and Gratin, 1994. An Efficient Algorithm for Drainage
        Networks Extraction on DEMs. Journal of Visual Communication and Image
        Representation, 5(2), 181-189
    .. [2] Liu et al., 2009. Another Fast and Simple DEM Depression-Filling
        Algorithm Based on Priority Queue Structure. Atmopsheric and Oceanic
        Science Letters, 2(4) 214-219

    """
    print('Outflow Point Flood Fill')

    # Rename or copy input so that input_array is a local variable?
    # input_array = np.copy(input_array)

    # Build mask of outflow point cells
    # Should we check if outflow cells are active?
    data_mask = np.isfinite(input_array)
    outflow_mask = np.zeros(data_mask.shape, dtype=np.bool)
    for out_row, out_col in outflow_pts:
        outflow_mask[out_row, out_col] = 1

    # Set h_max to a value larger than the array maximum to ensure
    #   that the while loop will terminate
    h_max = np.nanmax(input_array) + 10
    print('  h_max: {}'.format(h_max))

    # Initialize output array
    # Set masked/inactive cells to nan (these will be reset before returning)
    # Set active/non-outflow cells to h_max (these values will be "eroded")
    output_array = np.copy(input_array)
    output_array[~data_mask] = np.nan
    output_array[data_mask & (~outflow_mask)] = h_max

    # Build priority queue and place outflow pixels into priority queue
    # Last value is flag to indicate if cell is an outflow cell
    put = heapq.heappush
    get = heapq.heappop
    fill_heap = [
        (output_array[t_row, t_col], int(t_row), int(t_col), True)
        for t_row, t_col in np.transpose(np.where(outflow_mask))]
    heapq.heapify(fill_heap)

    def neighbors(row, col, four_way=False):
        """Return indices of adjacent cells"""
        if four_way:
            return [
                (row - 1, col), (row, col + 1),
                (row + 1, col), (row, col - 1)]
        else:
            return [
                (row - 1, col), (row - 1, col + 1),
                (row, col + 1), (row + 1, col + 1),
                (row + 1, col), (row + 1, col - 1),
                (row, col - 1), (row - 1, col - 1)]

    # Iterate until priority queue is empty
    while True:
        try:
            h_crt, t_row, t_col, outflow_flag = get(fill_heap)
        except IndexError:
            break
        for n_row, n_col in neighbors(t_row, t_col, four_way):
            # Skip cell if an outflow cell
            try:
                if outflow_mask[n_row, n_col] or not data_mask[n_row, n_col]:
                    continue
            except IndexError:
                continue
                
            if output_array[n_row, n_col] == h_max:
                output_array[n_row, n_col] = max(
                    h_crt, input_array[n_row, n_col])
                put(fill_heap, (output_array[n_row, n_col], n_row, n_col, False))
                
    # Reset masked/nodata values to their original value
    output_array[~data_mask] = input_array[~data_mask]
    
    return output_array

### Set inactive cells to nodata and compute fill

In [4]:
dem_fill = dem_adj.copy()
print(dem_fill)
dem_mask = (hru_type > 0)
dem_fill[~dem_mask] = np.nan

dem_fill = outflow_fill(dem_fill, outflow_pts, four_way=False)
dem_fill[~dem_mask] = dem_adj[~dem_mask]
print(dem_fill)

[[ 1.   1.   1.   1.   1.   1. ]
 [ 1.   0.1  1.   1.   1.   1. ]
 [ 0.1  0.6  0.2  0.3  1.   1. ]
 [ 0.2  0.7  0.3  0.4  0.1  1. ]
 [ 1.   0.8  0.4  1.   1.   1. ]
 [ 1.   1.   0.2  1.   1.   1. ]]
Outflow Point Flood Fill
  h_max: 11.0
[[ 1.   1.   1.   1.   1.   1. ]
 [ 1.   0.3  1.   1.   1.   1. ]
 [ 0.1  0.6  0.3  0.3  1.   1. ]
 [ 0.2  0.7  0.3  0.4  0.1  1. ]
 [ 1.   0.8  0.4  1.   1.   1. ]
 [ 1.   1.   0.4  1.   1.   1. ]]


In [5]:
dem_fill = dem_adj.copy()
print(dem_fill)
dem_mask = (hru_type > 0)
dem_fill[~dem_mask] = np.nan

dem_fill = outflow_fill(dem_fill, outflow_pts, four_way=True)
dem_fill[~dem_mask] = dem_adj[~dem_mask]
print(dem_fill)

[[ 1.   1.   1.   1.   1.   1. ]
 [ 1.   0.1  1.   1.   1.   1. ]
 [ 0.1  0.6  0.2  0.3  1.   1. ]
 [ 0.2  0.7  0.3  0.4  0.1  1. ]
 [ 1.   0.8  0.4  1.   1.   1. ]
 [ 1.   1.   0.2  1.   1.   1. ]]
Outflow Point Flood Fill
  h_max: 11.0
[[ 1.   1.   1.   1.   1.   1. ]
 [ 1.   0.6  1.   1.   1.   1. ]
 [ 0.1  0.6  0.4  0.4  1.   1. ]
 [ 0.2  0.7  0.4  0.4  0.1  1. ]
 [ 1.   0.8  0.4  1.   1.   1. ]
 [ 1.   1.   0.4  1.   1.   1. ]]
